# 作业1 第一项

## 2 Byte-Pair Encoding (BPE) Tokenizer


在作业的第一部分，我们将训练并实现一个基于字节的字节对编码（BPE）分词器。具体来说，我们将任意（Unicode）字符串表示为字节序列，并在该字节序列上训练我们的 BPE 分词器。之后，我们将使用这个分词器将文本（一个字符串）编码为标记（一个整数序列），用于语言建模。

### 2.1 Unicode 标准

Unicode 是一个将字符映射到整数代码点的文本编码标准。截至 Unicode 16.0（2024 年 9 月发布），该标准定义了 168 个脚本中的 154,998 个字符。例如，字符“s”的代码点是 115（通常表示为 U+0073，其中 U+是一个惯例前缀，0073 是十六进制的 115），字符“ ”的代码点是 29275。  
在 Python 中，你可以使用 ord()函数将单个 Unicode 字符代码点 转换为它的整数表示。chr()函数将一个整数 Unicode 代码点转换为对应字符的字符串。  
每一个字符都被赋予一个唯一的编号，称为代码点。格式为：  
U+XXXX（例如 U+0073 表示小写字母 s）  
代码点的数值范围为从 U+0000 到 U+10FFFF，共计 1,114,112 个可能的编码空间。不用担心不够用，远远大于所有语言的综合  
最新的unicode 已经支持emoji了

### 2.2 utf-8 unicode的编码形式
UTF = Unicode Transformation Format

是 Unicode 最常用的实现方式之一，它是一种可变长度字符编码格式，使用 1 到 4 个字节 来表示 Unicode 代码点。  
同时兼容 ASCII，又能表达全世界所有语言。
例如：
1. ASCII 字符 'A'  
Unicode 码点：U+0041 = 0x41 = 0100 0001  
范围：U+0000 ~ U+007F → 1 字节  
UTF-8 编码：01000001 → 0x41  
2. 汉字 '你'  
Unicode：U+4F60 = 0x4F60 = 01001111 01100000  
范围：U+0800 ~ U+FFFF → 3 字节  
UTF-8 编码：0xE4 0xBD 0xA0

In [11]:
#将整数代码点(记得从16进制转到10进制，例如 0041 -> 65) 转换为对应的字符
print(chr(20320))
print(chr(65))
chr(0)

你
A


'\x00'

In [15]:
# 将单个 Unicode 字符转换为对应的代码点（整数）10进制
print(ord('B'))
ord('牛')

66


29275

In [ ]:
# 在新的 cell 中运行以下代码来观察区别

char = chr(0)

print("字符本身:")
print(repr(char))  # __repr__() 方法的结果

print("\n打印表示:")
print(char)  # 直接打印的结果

print("\n在字符串中:")
print(f"字符在字符串中: '{char}'")

print("\n更多信息:")
print(f"长度: {len(char)}")
print(f"Unicode码点: {ord(char)}")
print(f"字节表示: {char.encode('utf-8')}")

字符本身:
'\x00'

打印表示:
 

在字符串中:
字符在字符串中: ' '

更多信息:
长度: 1
Unicode码点: 0
字节表示: b'\x00'


In [ ]:
test_string = "this is a test" + chr(0) + "string"

print("验证NULL字符确实存在:")
print(f"NULL字符在位置: {test_string.find(chr(0))}")
print(f"NULL字符前的部分: '{test_string[:14]}'")
print(f"NULL字符后的部分: '{test_string[15:]}'")

# 使用字节表示更清楚地看到
print(f"\n字节表示: {test_string.encode('utf-8')}")

验证NULL字符确实存在:
NULL字符在位置: 14
NULL字符前的部分: 'this is a test'
NULL字符后的部分: 'string'

字节表示: b'this is a test\x00string'


## problem

![alt text](img/image.png)

**Answer**  
a：'\x00'  
b：chr(0)的结果是'\x00'，(b) 该字符的字符串表示（repr()）与其打印表示有何不同？
chr(0) 是 NULL 字符，在大多数终端中不可见
repr() 的设计目的是让程序员能够看到对象的确切内容
print() 的目的是显示用户友好的输出  
c： this is a teststring ，在字符串中隐身了。

![alt text](image.png)

In [ ]:
test_string = "hello! こんにちは!"
utf8_encoded = test_string.encode("utf-8")
print(utf8_encoded)
# 字符串-》字节序列的转换
print(type(utf8_encoded))
list(utf8_encoded)
# list(字节序列) 会把每个字节变成十进制整数的列表，是查看 UTF-8 编码底层结构非常直观的方式

b'hello! \xe3\x81\x93\xe3\x82\x93\xe3\x81\xab\xe3\x81\xa1\xe3\x81\xaf!'
<class 'bytes'>


[104,
 101,
 108,
 108,
 111,
 33,
 32,
 227,
 129,
 147,
 227,
 130,
 147,
 227,
 129,
 171,
 227,
 129,
 161,
 227,
 129,
 175,
 33]

a: UTF-8 是变长编码，常见的英文和符号字符只用 1 个字节（ASCII 范围）。对于 Web 和常见文本，英文占比高，用 UTF-8 编码可以大大节省空间。在现代硬件和系统中已成为事实标准  
b:该函数错误地假设每个字节都是独立的UTF-8字符，但UTF-8使用可变长度编码，多字节字符必须作为完整序列一起解码。  
c:一个无法解码为合法 Unicode 字符的两字节序列示例是：0xC0 0xAF。   
解释：该序列属于 UTF-8 中的非法“过长编码”（overlong encoding），本应表示 /（U+002F），但使用了不必要的两字节形式，因此被 Unicode 标准禁止，不会被合法解码器接受。

## 2.3 Subword Tokenization

 1. 解决的问题:
    - 字节级分词: 序列太长，计算慢，难以捕获长期依赖
    - 词级分词: 存在未登录词问题，词表过大

 2. 子词分词优势:
    - 词级和字节级之间的平衡点
    - 用较大词表换取更好的压缩效果
    - 例如: 频繁出现的字节序列"the"可以作为单个token

 3. BPE (Byte-Pair Encoding):
    - 压缩算法，迭代合并最频繁出现的字节对
    - 构建过程称为"训练"BPE分词器
    - 通过添加子词单元最大化输入序列的压缩率

## 2.4 BPE Tokenizer Training

本文将 BPE 分词训练过程分为三步：

 1. 词表初始化（Vocabulary Initialization）
- **目标**：建立初始的 token → ID 映射表。  
- **做法**：采用“字节级 BPE”，将所有 256 个可能字节（0–255）作为初始 token。  
- **结果**：初始词表大小 = 256，每个字节对应一个 token。  

---

 1. 预分词（Pre-tokenization）
1. **为什么需要？**  
   - 避免每轮合并都遍历整个语料带来的高耗时。  
   - 限制跨“语义边界”合并，保证 `dog.` 与 `dog!` 等相近含义的单词共享子词。  
2. **常用正则（GPT-2）**：  
   ```python
   PAT = r"""
   '(?:[sdmt]|ll|ve|re)   # 英文缩写
   | ?\p{L}+              # 连续字母
   | ?\p{N}+              # 连续数字
   | ?[^\s\p{L}\p{N}]+    # 其他字符序列
   """
   ```  
3. **流程**：使用 `re.finditer()` 提取 pre-token，统计每个 pre-token 的出现频次。  

---

 1. BPE 合并（BPE Merges）
1. **输入**：预分词后的字节序列（不跨 pre-token 边界）  
2. **统计**：计算所有相邻字节对的共现频率  
3. **合并**：每轮选取出现频率最高的字节对 (A, B)，合并为新 token “AB”  
4. **更新**：将新 token 加入词表  
5. **迭代**：重复上述过程，直至达到目标词表大小  

 训练细节
- 初始词表：256 个单字节（0–255）  
- 最终词表大小 = 初始词表大小 + 合并轮次  
- **不允许跨 pre-token 边界合并**  
- **频率相同** 时，按字典序（lexicographic order）选择最大的 pair  

---

 1. 特殊 Token（Special Tokens）
- 在训练前或训练后，手动加入具有特殊含义的字符串，如：  
  - `<|endoftext|>`：文本结束标记  
  - `<pad>`、`<cls>`、`<sep>`：模型控制元信息  
- **特点**：  
  - 固定 ID，不参与任何拆分或合并  
  - 编码时作为独立 token 输出  

## Example (bpe_example): BPE training example


假设我们有以下训练语料：
low
lower
newest
widest

第一步：初始化词表（Tokenize by characters）
我们将每个单词拆分为字符，并在结尾添加特殊标记 </w> 表示词末：
l o w </w>  
l o w e r </w>  
n e w e s t </w>  
w i d e s t </w>

第二步：统计相邻字符对的频率
统计所有训练样本中，连续字符对出现的频率，例如：

e s: 出现 2 次（在 "newest", "widest" 中）

w </w>: 出现 1 次（在 "low" 中）

l o: 1 次

o w: 1 次

w e: 1 次

第三步：合并频率最高的字符对
假设 e s 是频率最高的对，我们将所有 e s 替换为一个新 token es，更新后的分词结果为：
l o w </w>  
l o w e r </w>  
n e w est </w>  
w i d est </w>

最终结果
重复上述过程直到达到预设的合并次数或词表大小为止。最终词表将包含如下 token：
l, o, w, </w>, e, r, n, i, d, s, t, es, est, ...


In [ ]:
b="你".encode("utf-8")


'你'